## Dashboard Creation with Plotly Dash

In [2]:
#pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [9]:
# This code would typically be in a separate .py file
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import sqlite3

# Connect to the database
conn = sqlite3.connect('time_series.db')
cursor = conn.cursor()

# Create the 'stock_prices' table if it doesn't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS stock_prices (
        date TEXT,
        ticker TEXT,
        open REAL,
        high REAL,
        low REAL,
        close REAL,
        volume INTEGER
    )
''')

# Insert some sample data into 'stock_prices'
sample_stock_data = [
    ('2022-10-27', 'AAPL', 170.00, 172.50, 168.80, 171.25, 100000),
    ('2023-10-28', 'AAPL', 171.50, 173.00, 170.20, 172.75, 120000),
    # Add more data as needed
]
cursor.executemany("INSERT INTO stock_prices VALUES (?, ?, ?, ?, ?, ?, ?)", sample_stock_data)

# Create the 'economic_indicators' table if it doesn't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS economic_indicators (
        date TEXT,
        indicator_name TEXT,
        value REAL
    )
''')

# Insert some sample data into 'economic_indicators'
sample_indicator_data = [
    ('2022-10-27', 'Interest Rate', 0.05),
    ('2023-10-28', 'Interest Rate', 0.052),
    # Add more data as needed
]
cursor.executemany("INSERT INTO economic_indicators VALUES (?, ?, ?)", sample_indicator_data)

conn.commit() # Commit changes to persist the table creation and data insertion

# Now load data, after ensuring tables exist
stocks_df = pd.read_sql_query("SELECT * FROM stock_prices", conn)
indicators_df = pd.read_sql_query("SELECT * FROM economic_indicators", conn)

# Initialize the Dash app
app = Dash(__name__)

# Define layout
app.layout = html.Div([
    html.H1("Financial Dashboard"),

    html.Div([
        html.Label("Select Ticker:"),
        dcc.Dropdown(
            id='ticker-dropdown',
            options=[{'label': ticker, 'value': ticker} for ticker in stocks_df['ticker'].unique()],
            value='AAPL'
        ),

        html.Label("Select Date Range:"),
        dcc.DatePickerRange(
            id='date-picker',
            start_date=stocks_df['date'].min(),
            end_date=stocks_df['date'].max(),
            max_date_allowed=stocks_df['date'].max(),
            min_date_allowed=stocks_df['date'].min()
        ),
    ]),

    html.Div([
        dcc.Graph(id='stock-price-graph')
    ]),

    html.Div([
        html.Label("Select Economic Indicator:"),
        dcc.Dropdown(
            id='indicator-dropdown',
            options=[{'label': ind, 'value': ind} for ind in indicators_df['indicator_name'].unique()],
            value='Interest Rate'
        ),
    ]),

    html.Div([
        dcc.Graph(id='economic-indicator-graph')
    ]),
])

# Define callback to update stock price graph
@app.callback(
    Output('stock-price-graph', 'figure'),
    [
        Input('ticker-dropdown', 'value'),
        Input('date-picker', 'start_date'),
        Input('date-picker', 'end_date')
    ]
)
def update_stock_graph(ticker, start_date, end_date):
    filtered_df = stocks_df[
        (stocks_df['ticker'] == ticker) &
        (stocks_df['date'] >= start_date) &
        (stocks_df['date'] <= end_date)
    ]

    fig = px.line(
        filtered_df,
        x='date',
        y='close',
        title=f'{ticker} Stock Price'
    )

    fig.update_layout(
        xaxis_title='Date',
        yaxis_title='Price ($)'
    )

    return fig

# Define callback to update economic indicator graph
@app.callback(
    Output('economic-indicator-graph', 'figure'),
    [
        Input('indicator-dropdown', 'value'),
        Input('date-picker', 'start_date'),
        Input('date-picker', 'end_date')
    ]
)
def update_indicator_graph(indicator, start_date, end_date):
    filtered_df = indicators_df[
        (indicators_df['indicator_name'] == indicator) &
        (indicators_df['date'] >= start_date) &
        (indicators_df['date'] <= end_date)
    ]

    fig = px.line(
        filtered_df,
        x='date',
        y='value',
        title=f'{indicator} Over Time'
    )

    fig.update_layout(
        xaxis_title='Date',
        yaxis_title='Value'
    )

    return fig

# If running this script directly
if __name__ == '__main__':
    app.run(debug=True) # Changed from app.run_server to app.run

# Note: In the notebook, this won't actually start the server
print("In a separate Python script, you would run this code to launch a Dash dashboard.")

<IPython.core.display.Javascript object>

In a separate Python script, you would run this code to launch a Dash dashboard.
